In [1]:
import os
import tensorflow as tf
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import CSVLogger


from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

In [2]:
# !pip install tensorflow-gpu

In [3]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [4]:
orig_dir=os.getcwd()
orig_dir

'C:\\Users\\Ying\\Downloads\\LBP_TOP'

In [5]:
# Retrieve Full HOF Data
features_data_hof_full=[]
features_label_hof_full=[]

path="C:/Users/Ying/Downloads/LBP_TOP/HOF_cropped_classified"
folders=os.listdir(path)
t_count=0
for f in folders:
    count=1
    folder_path=os.path.join(path, f)
    files=os.listdir(folder_path)
    for file in files:
        file_path=os.path.join(folder_path,file)
        features=os.listdir(file_path)
        for feature in features:
            feature_path=os.path.join(file_path,feature)
            histogram=np.load(feature_path)
            histogram = np.mean(histogram, axis=0)
            features_data_hof_full.append(histogram)
            features_label_hof_full.append(f)
            count=count+1
            t_count=t_count+1

In [6]:
x = features_data_hof_full
length = max(map(len, x))
features_data_hof_full = np.array([np.concatenate((xi,[0]*(length-len(xi)))) for xi in x])


In [7]:
# features_data_hof_full=np.reshape(features_data_hof_full,(10,15)).T
# print(features_data_hof_full.shape)
print(features_data_hof_full.shape)


(150, 14256)


In [8]:
# X = features_data_hof_full
# y = features_label_hof_full

In [9]:
# Retrieve Full LBP Data
features_data_l=[]
features_label_l=[]

path="C:/Users/Ying/Downloads/LBP_TOP/LBP_TOP_cropped_classified"
folders=os.listdir(path)
t_count=0
for f in folders:
    count=1
    folder_path=os.path.join(path, f)
    files=os.listdir(folder_path)
    for file in files:
        file_path=os.path.join(folder_path,file)
        features=os.listdir(file_path)
        for feature in features:
            feature_path=os.path.join(file_path,feature)
            histogram=np.load(feature_path)
            histogram = np.mean(histogram, axis=0)
            features_data_l.append(histogram)
            features_label_l.append(f)
            count=count+1
            t_count=t_count+1

In [10]:
# Process to correct format for DNN
features_data_l=np.concatenate(features_data_l, axis=1).T

print(features_data_l.shape)


(150, 768)


In [11]:
# X = features_data_l
# y = features_label_l

In [12]:
X = np.concatenate((features_data_hof_full,features_data_l),axis=1)
y = features_label_l

In [13]:
X.shape

(150, 15024)

In [14]:
# One hot encoder of labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
dummy_y = np_utils.to_categorical(integer_encoded)

In [15]:
save_location='C:/Users/Ying/Downloads/LBP_TOP/Run_time_data'
modelname= 'RB_FF_NN13'
savelocation=os.path.join(save_location, modelname)
filepath=savelocation + ".hdf5"
csv_logger=CSVLogger(savelocation +'.csv')
#callbacks_list  = [checkpoint,csv_logger,LRScheduler]

In [16]:
model = Sequential()
model.add(Dense(512, activation='sigmoid', input_shape=(X.shape[1],)))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(4, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               7692800   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
Total params: 8,482,820
Trainable params: 8,482,820
Non-trainable params: 0
_________________________________________________________________


In [17]:
callbacks=[csv_logger, ModelCheckpoint(filepath,monitor='val_loss', save_best_only=True)]

In [18]:
opt='RMSProp'
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.20)
history = model.fit(X_train, y_train, batch_size=2, epochs=200, verbose=1,validation_split=0.005, callbacks=callbacks)

Epoch 1/200
60/60 [==============================] - 4s 57ms/step - loss: 1.6103 - accuracy: 0.3277 - val_loss: 1.5028 - val_accuracy: 0.0000e+00
Epoch 2/200
60/60 [==============================] - 4s 59ms/step - loss: 1.4398 - accuracy: 0.3697 - val_loss: 0.5706 - val_accuracy: 1.0000
Epoch 3/200
60/60 [==============================] - 3s 58ms/step - loss: 1.3648 - accuracy: 0.3950 - val_loss: 1.3516 - val_accuracy: 0.0000e+00
Epoch 4/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3651 - accuracy: 0.3277 - val_loss: 0.1593 - val_accuracy: 1.0000
Epoch 5/200
60/60 [==============================] - 3s 57ms/step - loss: 1.4035 - accuracy: 0.3782 - val_loss: 0.8665 - val_accuracy: 1.0000
Epoch 6/200
60/60 [==============================] - 3s 57ms/step - loss: 1.4229 - accuracy: 0.3613 - val_loss: 1.0278 - val_accuracy: 0.0000e+00
Epoch 7/200
60/60 [==============================] - 3s 57ms/step - loss: 1.4269 - accuracy: 0.3277 - val_loss: 0.8208 - val_accuracy: 1

Epoch 58/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3944 - accuracy: 0.3445 - val_loss: 1.2173 - val_accuracy: 0.0000e+00
Epoch 59/200
60/60 [==============================] - 3s 57ms/step - loss: 1.4025 - accuracy: 0.3613 - val_loss: 1.1505 - val_accuracy: 0.0000e+00
Epoch 60/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3828 - accuracy: 0.4034 - val_loss: 0.9132 - val_accuracy: 1.0000
Epoch 61/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3818 - accuracy: 0.4118 - val_loss: 1.2455 - val_accuracy: 0.0000e+00
Epoch 62/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3706 - accuracy: 0.3529 - val_loss: 0.8717 - val_accuracy: 1.0000
Epoch 63/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3548 - accuracy: 0.4118 - val_loss: 0.7499 - val_accuracy: 1.0000
Epoch 64/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3480 - accuracy: 0.4202 - val_loss: 0.7981 - val_accu

60/60 [==============================] - 3s 58ms/step - loss: 1.3267 - accuracy: 0.4202 - val_loss: 0.7932 - val_accuracy: 1.0000
Epoch 116/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3294 - accuracy: 0.4202 - val_loss: 0.8372 - val_accuracy: 1.0000
Epoch 117/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3278 - accuracy: 0.4202 - val_loss: 0.8196 - val_accuracy: 1.0000
Epoch 118/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3311 - accuracy: 0.4202 - val_loss: 0.8187 - val_accuracy: 1.0000
Epoch 119/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3323 - accuracy: 0.4202 - val_loss: 0.8924 - val_accuracy: 1.0000
Epoch 120/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3294 - accuracy: 0.4202 - val_loss: 0.8519 - val_accuracy: 1.0000
Epoch 121/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3250 - accuracy: 0.4202 - val_loss: 0.8558 - val_accuracy: 1.0000
Epoch 

Epoch 172/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3240 - accuracy: 0.4202 - val_loss: 0.8810 - val_accuracy: 1.0000
Epoch 173/200
60/60 [==============================] - 3s 58ms/step - loss: 1.3251 - accuracy: 0.4202 - val_loss: 0.8483 - val_accuracy: 1.0000
Epoch 174/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3240 - accuracy: 0.4202 - val_loss: 0.9013 - val_accuracy: 1.0000
Epoch 175/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3221 - accuracy: 0.4202 - val_loss: 0.9039 - val_accuracy: 1.0000
Epoch 176/200
60/60 [==============================] - 3s 57ms/step - loss: 1.3229 - accuracy: 0.4202 - val_loss: 0.8239 - val_accuracy: 1.0000
Epoch 177/200
60/60 [==============================] - 3s 58ms/step - loss: 1.3209 - accuracy: 0.4202 - val_loss: 0.7849 - val_accuracy: 1.0000
Epoch 178/200
60/60 [==============================] - 3s 58ms/step - loss: 1.3260 - accuracy: 0.4202 - val_loss: 0.8797 - val_accuracy:

In [19]:
yhat_classes = model.predict_classes(X_test, verbose=0)
y_test_original = y_test.argmax(1)
y_train_original = y_train.argmax(1)
labels = np.unique(y_test_original)
a= confusion_matrix(y_test_original, yhat_classes)
print(a)
print(classification_report(y_test_original, yhat_classes))

[[12  0  0  0]
 [ 5  0  0  0]
 [ 5  0  0  0]
 [ 8  0  0  0]]
              precision    recall  f1-score   support

           0       0.40      1.00      0.57        12
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         8

    accuracy                           0.40        30
   macro avg       0.10      0.25      0.14        30
weighted avg       0.16      0.40      0.23        30



C:\Users\Ying\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
svm_model = svm.SVC(kernel = 'linear', C = 2,decision_function_shape='ovo').fit(X_train,y_train_original)

y_pred = svm_model.predict(X_test)

print(confusion_matrix(y_test_original, y_pred))

print("train score:",svm_model.score(X_train,y_train_original))
print("test score:",svm_model.score(X_test,y_test_original))

[[11  0  1  0]
 [ 3  1  1  0]
 [ 1  0  3  1]
 [ 3  2  1  2]]
train score: 1.0
test score: 0.5666666666666667
